In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import os

In [2]:
results = []
for k in range(100):
    if k==50:
        continue
    data = pd.read_csv(f"D:/ReClusterData/reclustered{k}_data.csv")  
    summations_per_label = np.zeros(20)
    # Initialize a 20x20 matrix to store correlation coefficients
    correlation_matrix = np.zeros((20, 20))
    
    # Iterate over each pair of clusters
    for i in range(20):
        summation=0
        for j in range(20):
            # Extract 'Serving RSSI_4G' values for clusters i and j
            cluster_i_values = data[data['Cluster_Label'] == i]['Serving RSSI_4G']
            cluster_j_values = data[data['Cluster_Label'] == j]['Serving RSSI_4G']
            
            # Check if both arrays have the same length
            min_len = min(len(cluster_i_values), len(cluster_j_values))
            if min_len > 0:
                # Truncate arrays to the same length
                cluster_i_values = cluster_i_values[:min_len]
                cluster_j_values = cluster_j_values[:min_len]
                
                # Calculate the correlation coefficient between the truncated values of clusters i and j
                correlation_matrix[i, j] = np.corrcoef(cluster_i_values, cluster_j_values)[0, 1]
            else:
                # If one of the arrays is empty, set correlation coefficient to NaN
                correlation_matrix[i, j] = 0
    
            summation += correlation_matrix[i, j]
        summations_per_label[i] = summation
    
    correlation_matrix = np.nan_to_num(correlation_matrix, nan=0.0)

    def find_primary_secondary(co_matrix):
        primary_sum=0
        primary_node=0
        secondary_node=[]
        for i in range(20):
            sum=0
    
            for j in range(20):
                sum+=co_matrix[i][j]
            
            if(sum>primary_sum ):
                primary_sum=sum
                primary_node=i
        
        for i in range(20):
            if(co_matrix[primary_node][i]>=0.3 and primary_node!=i):
                secondary_node.append(i)
    
        return primary_node,secondary_node

    def delete_secoondary_primary(co_matrix,primary_node,secondary_node):
        n=len(secondary_node)
        for i in range(20):
            co_matrix[primary_node][i]=0
            co_matrix[i][primary_node]=0
            for j in range(n):
                ele=secondary_node[j]
                #print(ele)
                co_matrix[ele][i]=0
                co_matrix[i][ele]=0
        return co_matrix

    print()
    print(f"for matrix {k}:")
    print()
    c=0
    for i in range(20):
        p_node,s_node=find_primary_secondary(correlation_matrix)
        if any(item == 0 for item in s_node):
            c=c+1
        if p_node==0:
            c=c+1
        if c==2:
            break
        correlation_matrix=delete_secoondary_primary(correlation_matrix,p_node,s_node)
        print('Primary Node:',p_node)
        if s_node:
            print('Secondary Node:',s_node)
            
        # Calculate the median of 'Longitude', 'Latitude', 'Serving RSSI_4G' for the primary node
        primary_data = data[data['Cluster_Label'] == p_node]
        median_longitude = primary_data['Longitude'].median()
        median_latitude = primary_data['Latitude'].median()
        median_rssi = primary_data['Serving RSSI_4G'].median()

        # Append the results to the list
        results.append({
            'slNo': len(results) + 1,
            'Cluster': k,
            'Primary Node': p_node,
            'Median Longitude': median_longitude,
            'Median Latitude': median_latitude,
            'Median RSSI': median_rssi
        })


for matrix 0:

Primary Node: 5
Secondary Node: [0, 7, 8, 9, 10, 11, 13, 14, 18]
Primary Node: 17
Secondary Node: [6, 15, 16]
Primary Node: 4
Primary Node: 19
Primary Node: 12
Primary Node: 3
Primary Node: 1
Primary Node: 2

for matrix 1:

Primary Node: 10
Secondary Node: [5, 7, 14, 15, 16]
Primary Node: 18
Secondary Node: [2, 9]
Primary Node: 4
Secondary Node: [0, 1, 8]
Primary Node: 11
Secondary Node: [17]
Primary Node: 19
Secondary Node: [6]
Primary Node: 12
Primary Node: 3
Primary Node: 13

for matrix 2:

Primary Node: 3
Primary Node: 19
Secondary Node: [4]
Primary Node: 11
Secondary Node: [0, 7]
Primary Node: 10
Primary Node: 12
Primary Node: 13
Primary Node: 15
Primary Node: 1
Primary Node: 5
Primary Node: 8
Primary Node: 16
Primary Node: 9
Primary Node: 17
Primary Node: 14
Primary Node: 6
Primary Node: 2
Primary Node: 18

for matrix 3:

Primary Node: 8
Secondary Node: [6, 17, 18]
Primary Node: 3
Secondary Node: [1, 9, 15, 16]
Primary Node: 4
Primary Node: 7
Secondary Node: [5, 1

C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]



for matrix 17:

Primary Node: 3
Secondary Node: [0, 1, 5, 7, 8, 9, 10, 13, 14, 15, 16, 19]
Primary Node: 6
Secondary Node: [2, 11, 12, 17]
Primary Node: 4

for matrix 18:

Primary Node: 12
Secondary Node: [3, 7, 8, 10, 13, 14, 15, 16, 17, 18]
Primary Node: 2
Secondary Node: [4, 19]
Primary Node: 0
Secondary Node: [1]
Primary Node: 5
Primary Node: 11
Primary Node: 9
Primary Node: 6

for matrix 19:

Primary Node: 6
Secondary Node: [1, 3, 7, 8]
Primary Node: 19
Secondary Node: [9, 13, 15, 17, 18]
Primary Node: 0
Secondary Node: [12]
Primary Node: 16
Primary Node: 10
Primary Node: 5
Primary Node: 2
Primary Node: 4
Primary Node: 14
Primary Node: 11

for matrix 20:

Primary Node: 2
Secondary Node: [0, 3, 7, 10, 11, 13, 16, 19]
Primary Node: 4
Secondary Node: [6, 9, 12, 18]
Primary Node: 15
Secondary Node: [1, 14]
Primary Node: 8
Primary Node: 5
Primary Node: 17

for matrix 21:

Primary Node: 4
Primary Node: 0
Secondary Node: [7, 13, 16]
Primary Node: 19
Secondary Node: [8, 14, 17]
Primary N

C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)



for matrix 23:

Primary Node: 1
Secondary Node: [0, 5, 6, 8, 9, 10, 16, 18]
Primary Node: 13
Secondary Node: [2, 7, 11, 17]
Primary Node: 12
Secondary Node: [14, 19]
Primary Node: 4
Primary Node: 15

for matrix 24:

Primary Node: 18
Secondary Node: [0, 1, 2, 3, 4, 7, 9, 15, 16, 19]
Primary Node: 10
Secondary Node: [13]
Primary Node: 8
Secondary Node: [6]
Primary Node: 14
Secondary Node: [12]
Primary Node: 17
Primary Node: 5
Primary Node: 11

for matrix 25:

Primary Node: 14
Secondary Node: [5, 6, 8, 9, 16]
Primary Node: 4
Secondary Node: [0, 11, 19]
Primary Node: 15
Secondary Node: [7]
Primary Node: 10
Secondary Node: [12, 13]
Primary Node: 1
Secondary Node: [2, 3]
Primary Node: 17
Primary Node: 18

for matrix 26:

Primary Node: 3
Secondary Node: [4, 8, 16]
Primary Node: 0
Secondary Node: [10, 12, 17]
Primary Node: 11
Secondary Node: [15, 19]
Primary Node: 2
Primary Node: 14
Secondary Node: [6]
Primary Node: 9
Secondary Node: [5]
Primary Node: 1
Primary Node: 7
Primary Node: 13
Primar

...


for matrix 97:

Primary Node: 10
Secondary Node: [0, 3, 4, 8, 13, 14, 16, 17, 18]
Primary Node: 5
Secondary Node: [9, 12]
Primary Node: 11
Secondary Node: [6, 15]
Primary Node: 1


C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)



for matrix 98:

Primary Node: 12
Secondary Node: [0, 1, 3, 6, 7, 8, 11, 18]
Primary Node: 9
Secondary Node: [4, 5, 13, 16, 17, 19]
Primary Node: 10
Secondary Node: [2, 15]


C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]



for matrix 99:

Primary Node: 7
Secondary Node: [0, 4, 8, 9, 10, 17]
Primary Node: 16
Secondary Node: [1, 12, 14]
Primary Node: 5
Secondary Node: [6, 15]
Primary Node: 19
Secondary Node: [2, 18]
Primary Node: 3
Primary Node: 11


In [ ]:
results_df = pd.DataFrame(results)
file_path = Path("D:/PathPlanning_Data.csv")
results_df.to_csv(file_path, index=False)